In [5]:
import pandas as pd
from pandas import DataFrame as df
import xml.etree.ElementTree as ET

In [6]:
buy_from_customer = df(columns=["date",
                        "phone_number",
                        "notebook",
                        "tablet",
                        "phone",
                        "spare_part",
                        "conversation",
                        "customer_price",
                        "our_price",
                        "recycle",
                        "customer_decision"])
buy_from_customer

,date,phone_number,notebook,tablet,phone,spare_part,conversation,customer_price,our_price,recycle,customer_decision


In [54]:
tree = ET.parse('output.xml')
root = tree.getroot()
r = root.findall("Lead")

def make_dict_of_leads(tree=tree):
    root = tree.getroot()
    r = root.findall("Lead")

    dict_of_leads = dict();
    for lead in root.iter('Lead'):
        id = lead.attrib["id"]

        pulpy = ET.fromstring(lead.attrib["text"])
        source_sentence = pulpy.find("b").find("s");
        text = "";
        for i in source_sentence.itertext():
            text += i;
        dict_of_leads[id] = text;
    return dict_of_leads;



[]

In [89]:



for k in make_dict_of_leads().keys():
    print(k, dict_of_leads[k])

46 89140580517 продажа айфон6s на выезде Ляпидевского 12.
24 / / Клиент продаёт нам ноут.
53 89140580517 замена клавиатуры Acer Aspire V3-571G.
18 / / Ломбард.
10 89140580517 куп 17 зац.
28 89140580517 продает Асер 2234 - отказ слишком старый.
54 89140580517 модуль памяти.
30 89140580517 прод Асер офисный предложил 3000 зацепка.
3 89140580517 уу ноутбук купить за 17000 руб.
19 89140580517 ломб ноут Асер 5733 наша цена 5000.
55 89140580517 жесткий диск купить 320.
21 89140580517 ломб Асер 5733 наша цена 5000.
29 89140580517 продаст Асер офисный предложил 3000 зацепка.
6 89140580517 куп тел 27.
22 89140580517 ломб сони хперия з3.
7 89140580517 куп телефон 27.
40 89140580517 продажа т samsung а5 на выезде Ляпидевского 12.
49 89140580517 перевод звонка в Якутию спросить покупаем ли бензопилы.
16 89140580517 buy 17 зац.
25 89140580517 продажа ноут Асер 2234 - ( наш отказ | мы отказались | отказал ) слишком старый.
31 89140580517 прод Асер офисный предложил 3000 зацепка.
56 89140580517 ватса